In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

In [ ]:
Data_set= pd.read_csv("/home/effrancodelos/Deep_learnig/Autoencoder_Notebook/Nova_data_agosto_2020/GBM_mRNA_miRNA_DNA.csv")
Data_set.head(n=20)


In [ ]:
#Create a data-set with the string columns

Pacients = Data_set['Pacients']




In [ ]:
#Remove string columns
Data_set.drop(['Pacients'], axis=1, inplace=True)

In [ ]:
#Data normalization 

normalized_df = minmax_scale(Data_set, axis = 0)
#normalized_df = StandardScaler().fit_transform(Data_set)

In [ ]:
normalized_df

In [ ]:
#split the data 80% training and 20% teste
    
X_train, X_val = train_test_split(normalized_df, test_size=0.2,random_state = seed(7))

In [ ]:
#Input feauter number 
ncol = normalized_df.shape[1]





In [1]:
encoding_dim = 100

In [ ]:
#Network configuration

input_dim = Input(shape = (ncol, ))
# Encoder Layers
encoded1 = Dense(500, activation = 'tanh')(input_dim)
encoded2 = Dense(encoding_dim, activation = 'tanh')(encoded1)

# Decoder Layers

decoded1 = Dense(500, activation = 'tanh')(encoded2)
decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)

In [ ]:
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded2)

In [ ]:
# Compile the Model
#autoencoder.compile(optimizer = 'adadelta', loss = 'binary_crossentropy')
autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error')# BEST

In [ ]:
autoencoder.summary()

In [ ]:
#fit the model

autoencoder.fit(X_train, X_train, epochs = 70, batch_size = 256, shuffle = False, validation_data = (X_val, X_val))

In [ ]:
#Obtain the reduced data

encoder = Model(inputs = input_dim, outputs = encoded2)
encoded_input = Input(shape = (encoding_dim, ))

In [ ]:
encoded_train = pd.DataFrame(encoder.predict(normalized_df))
encoded_train = encoded_train.add_prefix('feature_')

In [ ]:
encoded_train['Pacients'] = Pacients
#encoded_train['Radiations'] = Radiation

In [ ]:
#output dataset
print(encoded_train.shape)
encoded_train.head()

In [ ]:
encoded_train.to_csv("/home/effrancodelos/Deep_learnig/Autoencoder_Notebook/Nova_data_agosto_2020/GBM_mRNA_miRNA_DNA_Vanilla.csv", index=False)